要正确理解误差反向传播法，我个人认为有两种方法：一种是基于数学式；另一种是基于计算图

## 计算图
用计算图解题的情况下，需要按如下流程进行。
1.构建计算图。
2.在计算图上，从左向右进行计算。

这里的第2歩“从左向右进行计算”是一种正方向上的传播，简称为**正向传播**（forward propagation）。
**反向传播**即以反方向传播

使用计算图最大的原因是，可以通过反向传播高效计算导数

计算图的反向传播：沿着与正方向相反的方向，乘上局部导数

### 链式法则
> 如果某个函数由复合函数表示，则该复合函数的导数可以用构成复合函数的各个函数的导数的乘积表示。

如果
$$
z = t^2\\
t = x + y
$$
那么：
$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = 2t=2(x + y)
$$

---
反向传播是基于链式法则的:

![](./%E5%9B%BE5-7%20%E9%93%BE%E5%BC%8F%E6%B3%95%E5%88%99%E4%B8%8E%E8%AE%A1%E7%AE%97%E5%9B%BE.png)

![](./%E5%9B%BE5-8%20%E9%93%BE%E5%BC%8F%E6%B3%95%E5%88%99%E4%B8%8E%E8%AE%A1%E7%AE%97%E5%9B%BE.png)




## 反向传播
加法节点的反向传播将上游的值原封不动地输出到下游  
乘法的反向传播会乘以输入信号的翻转值，乘法的反向传播需要正向传播时的输入信号值。因此，实现乘法节点的反向传播时，要保存正向传播的输入信号


## 乘法层和加法层的实现

In [ ]:
# 乘法层 需要记住输入
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None

  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y
    return out

  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x
    return dx, dy

# 加法层 不需要记住输入
class AddLayer:
  def __init__(self):
    pass

  def forward(self, x, y):
    out = x + y
    return out

  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy


## 激活函数层的实现
ReLU函数
$$
\begin{aligned}
h(x) &=
\begin{cases}
x & (x\gt 0) \\
0 & (x\leqslant 0) \\
\end{cases}
\end{aligned}
$$

$$
\frac{\partial y}{\partial x}=
\begin{cases}
1 & (x \gt 0)\\
0 & (x \leqslant 0)\\
\end{cases}

$$

In [ ]:
class Relu:
  def __init__(self):
    self.mask = None
  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0
    return out
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    return dx


Sigmoid层：

$$f(x)=\frac{1}{1+e^{-x}}$$

设$u=1+e^{-x}$
$$
\begin{aligned}
f'(x)&=f'(u)u' \\
&=-\frac{1}{u^2}(-e^{-x}) \\
&=\frac{e^{-x}}{(1+e^{-x})^2}\\
&=\frac{1}{1+e^{-x}}\cdot\frac{e^{-x}}{1+e^{-x}}\\
&=\frac{1}{1+e^{-x}}\cdot\frac{1+e^{-x}-1}{1+e^{-x}}\\
&=\frac{1}{1+e^{-x}}\cdot(1-\frac{1}{1+e^{-x}})\\
&=f(x)(1-f(x))
\end{aligned}
$$

In [2]:
import numpy as np

class Sigmoid:
  def __init__(self):
    self.out = None
  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out
    return out
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    return dx


Affine层

![](./%E5%9B%BE5-24%20Affine%E5%B1%82%E8%AE%A1%E7%AE%97%E5%9B%BE.png)

$$
\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} W^T \\

\frac{\partial L}{\partial W} =  X^T\frac{\partial L}{\partial Y} \\

$$

In [3]:
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None
  def forward(self, x):
    self.x = x
    out = np.dot(x, self.W) + self.b
    return out
  def backward(self, dout):
    dx = np.dot(dout, self.W.T)
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0)
    return dx


Softmax-with-Loss 层（Softmax函数和交叉熵误差）
> 神经网络中进行的处理有推理（inference）和学习两个阶段。神经网络的推理通常不使用 Softmax层。比如，用图 5-28的网络进行推理时，会将最后一个 Affine层的输出作为识别结果。神经网络中未被正规化的输出结果（图 5-28中 Softmax层前面的 Affine层的输出）有时被称为“得分”。也就是说，当神经网络的推理只需要给出一个答案的情况下，因为此时只对得分最大值感兴趣，所以不需要 Softmax层。不过，神经网络的学习阶段则需要 Softmax层。

![](./%E5%9B%BE5-30%E3%80%80%E2%80%9C%E7%AE%80%E6%98%93%E7%89%88%E2%80%9D%E7%9A%84Softmax-with-Loss%E5%B1%82%E7%9A%84%E8%AE%A1%E7%AE%97%E5%9B%BE.png)


> 使用交叉熵误差作为 softmax函数的损失函数后，反向传播得到$(y1 − t1, y2 − t2, y3 − t3)$ 这样“漂亮”的结果。实际上，这样“漂亮”的结果并不是偶然的，而是为了得到这样的结果，特意设计了交叉熵误差函数。回归问题中输出层使用“恒等函数”，损失函数使用“平方和误差”，也是出于同样的理由（3.5节）。也就是说，使用“平方和误差”作为“恒等函数”的损失函数，反向传播才能得到$(y1 −t1, y2 − t2, y3 − t3)$这样“漂亮”的结果。

In [ ]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None  # 损失
    self.y = None  # softmax的输出
    self.t = None  # 监督数据（one-hot vector）

  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)
    return self.loss

  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size
    return dx


def softmax(x):
  c = np.max(x)
  exp_x = np.exp(x - c)
  sum_exp_x = np.sum(exp_x)
  return exp_x / sum_exp_x


def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)
  batch_size = y.shape[0]
  return -np.sum(t * np.log(y + 1e-7)) / batch_size


**前提**
神经网络中有合适的权重和偏置，调整权重和偏置以便拟合训练数据的
过程称为学习。神经网络的学习分为下面4个步骤。
**步骤1（mini-batch）**
从训练数据中随机选择一部分数据。
**步骤2（计算梯度）**
计算损失函数关于各个权重参数的梯度。
**步骤3（更新参数）**
将权重参数沿梯度方向进行微小的更新。
**步骤4（重复）**
重复步骤1、步骤2、步骤3

之前介绍的误差反向传播法会在步骤2中出现。
误差反向传播法可以快速高效地计算梯度。
